# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [Agent('Chef', 'Takes in dish names and comes up with ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them according to price'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them')
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in <list_of_dish_names> and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value, type: dict"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [Agent('Chef', 'Takes in dish names and comes up with ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them according to price'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

Subtask identified: Generate interesting Italian dish names and descriptions using Creative Writer.
Calling function Creative Writer with parameters {'instruction': 'Italian cuisine'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes
Getting LLM to perform the following task: Research popular Italian dishes
> 1. Dish Name: Spaghetti Carbonara Description: Classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese.  2. Dish Name: Margherita Pizza Description: Traditional pizza topped with tomato sauce, mozzarella cheese, and fresh basil.  3. Dish Name: Osso Buco Description: Braised veal shanks served with a flavorful sauce and gremolata.  4. Dish Name: Tiramisu Description: Decadent dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese.  5. Dish Name: Risotto ai Funghi Description: Creamy risotto cooked with mushrooms, onions, white wine, and Parmesan cheese.

Subtask identified: Generate interesting

In [9]:
output = my_agent.reply_user()

{'Menu Items': [{'Name': 'Spaghetti Carbonara', 'Description': 'Classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese.', 'Ingredients': 'Spaghetti pasta, eggs, pancetta (Italian bacon), Pecorino Romano cheese, black pepper', 'Price': '$5'}, {'Name': 'Margherita Pizza', 'Description': 'Traditional pizza topped with tomato sauce, mozzarella cheese, and fresh basil.', 'Ingredients': 'Dough, tomato sauce, mozzarella cheese, fresh basil', 'Price': '$5'}, {'Name': 'Osso Buco', 'Description': 'Braised veal shanks served with a flavorful sauce and gremolata.', 'Ingredients': 'Veal shanks, vegetables, broth, gremolata', 'Price': '$7'}, {'Name': 'Tiramisu', 'Description': 'Decadent dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese.', 'Ingredients': 'Ladyfingers, coffee, mascarpone cheese, cocoa powder', 'Price': '$10'}, {'Name': 'Gnocchi alla Sorrentina', 'Description': 'Soft potato dumplings baked with tomato sauce, mozzarella cheese, and fresh bas

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
1. Dish Name: Spaghetti Carbonara Description: Classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese.  2. Dish Name: Margherita Pizza Description: Traditional pizza topped with tomato sauce, mozzarella cheese, and fresh basil.  3. Dish Name: Osso Buco Description: Braised veal shanks served with a flavorful sauce and gremolata.  4. Dish Name: Tiramisu Description: Decadent dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese.  5. Dish Name: Risotto ai Funghi Description: Creamy risotto cooked with mushrooms, onions, white wine, and Parmesan cheese.

Subta

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes
Getting LLM to perform the following task: Research popular Italian dishes
> As an agent named Creative Writer, I can generate interesting dish names and descriptions for Italian cuisine. Here are 5 menu items: 1. Bella Risotto - Creamy risotto with saffron, mushrooms, and parmesan cheese. 2. Amore Alla Carbonara - Spaghetti carbonara with crispy pancetta, eggs, and pecorino cheese. 3. Dolce Vita Pizza - Thin-crust pizza topped with fresh tomatoes, mozzarella, basil, and prosciutto. 4. Sapore di Mare Linguine - Linguine pasta with a seafood medley of shrimp, clams, and mussels in a white wine sauce. 5. Tiramisu Affogato - Classic tiramisu dessert served with a shot of espresso poured over the top.

Subtask identified: Generate 5 Italian dish names and descriptions
Getting LLM

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
As an agent named Creative Writer, I can generate interesting dish names and descriptions for Italian cuisine. Here are 5 menu items: 1. Bella Risotto - Creamy risotto with saffron, mushrooms, and parmesan cheese. 2. Amore Alla Carbonara - Spaghetti carbonara with crispy pancetta, eggs, and pecorino cheese. 3. Dolce Vita Pizza - Thin-crust pizza topped with fresh tomatoes, mozzarella, basil, and prosciutto. 4. Sapore di Mare Linguine - Linguine pasta with a seafood medley of shrimp, clams, and mussels in a white wine sauce. 5. Tiramisu Affogato - Classic tiramisu dessert served with a shot of espre

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: List out the ingredients for each dish
Getting LLM to perform the following task: List out the ingredients for each dish
> 1. Rustic Polenta Parmigiana - Ingredients: polenta, marinara sauce, mozzarella, fresh basil. 2. Truffle Gnocchi al Tartufo - Ingredients: potato gnocchi, truffle cream sauce, Parmigiano Reggiano. 3. Caprese Arancini - Ingredients: risotto, mozzarella, cherry tomatoes, basil, marinara sauce. 4. Zesty Puttanesca Penne - Ingredients: penne pasta, spicy tomato sauce, olives, capers, anchovies. 5. Limoncello Tiramisu - Ingredients: ladyfingers, mascarpone cheese, eggs, sugar, limoncello liqueur, lemon zest.

Task completed successfully!

### End of Inner Agent: Chef ###


In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Use the dish_price function to assign prices to the 5 Italian dishes.
Calling function dish_price with parameters {'list_of_dish_names': ['Rustic Polenta Parmigiana', 'Truffle Gnocchi al Tartufo', 'Caprese Arancini', 'Zesty Puttanesca Penne', 'Limoncello Tiramisu']}
> {'Price': {'Rustic Polenta Parmigiana': '$2', 'Truffle Gnocchi al Tartufo': '$10', 'Caprese Arancini': '$4', 'Zesty Puttanesca Penne': '$2', 'Limoncello Tiramisu': '$9'}}

Task completed successfully!

### End of Inner Agent: Economist ###


In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why'}

### Start of Inner Agent: Boss ###
Subtask identified: Curate the top 3 Italian dishes based on price
Getting LLM to perform the following task: Curate the top 3 Italian dishes based on price
> Top 3 Italian dishes based on price are: Truffle Gnocchi al Tartufo, Limoncello Tiramisu, Caprese Arancini.

Task completed successfully!

### End of Inner Agent: Boss ###


In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Generate the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Dish Name: Truffle Gnocchi al Tartufo, Dish Description: Potato gnocchi in a decadent truffle cream sauce, garnished with Parmigiano Reggiano, Dish Ingredients: potato gnocchi, truffle cream sauce, Parmigiano Reggiano, Dish Price: $10
Dish Name: Limoncello Tiramisu, Dish Description: A refreshing twist on the classic tiramisu, infused with limoncello liqueur and topped with lemon zest, Dish Ingredients: ladyfingers, mascarpone cheese, eggs, sugar, limoncello liqueur, lemon zest, Dish Price: $9
Dish Name: Caprese Arancini, Dish Description: Crispy risotto balls stuffed with mozzarella, cherry tomatoes, and basil, served with marinara dipping sauce, Dish Ingredients: risotto, mozzarella, cherry tomatoes, basil, marinara sauce, Dish Price: $4


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
As an agent named Creative Writer, I can generate interesting dish names and descriptions for Italian cuisine. Here are 5 menu items: 1. Bella Risotto - Creamy risotto with saffron, mushrooms, and parmesan cheese. 2. Amore Alla Carbonara - Spaghetti carbonara with crispy pancetta, eggs, and pecorino cheese. 3. Dolce Vita Pizza - Thin-crust pizza topped with fresh tomatoes, mozzarella, basil, and prosciutto. 4. Sapore di Mare Linguine - Linguine pasta with a seafood medley of shrimp, clams, and mussels in a white wine sauce. 5. Tiramisu Affogato - Classic tiramisu dessert served with a shot of espre

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Dish Name: Truffle Gnocchi al Tartufo, Description: Potato gnocchi in a decadent truffle cream sauce, garnished with Parmigiano Reggiano, Ingredients: potato gnocchi, truffle cream sauce, Parmigiano Reggiano, Price: $10
Dish Name: Limoncello Tiramisu, Description: A refreshing twist on the classic tiramisu, infused with limoncello liqueur and topped with lemon zest, Ingredients: ladyfingers, mascarpone cheese, eggs, sugar, limoncello liqueur, lemon zest, Price: $9
Dish Name: Caprese Arancini, Description: Crispy risotto balls stuffed with mozzarella, cherry tomatoes, and basil, served with marinara dipping sauce, Ingredients: risotto, mozzarella, cherry tomatoes, basil, marinara sauce, Price: $4


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
Dish Name: Truffle Gnocchi al Tartufo, Description: Potato gnocchi in a decadent truffle cream sauce, garnished with Parmigiano Reggiano, Ingredients: potato gnocchi, truffle cream sauce, Parmigiano Reggiano, Price: $10
Dish Name: Limoncello Tiramisu, Description: A refreshing twist on the classic tiramisu, infused with limoncello liqueur and topped with lemon zest, Ingredients: ladyfingers, mascarpone cheese, eggs, sugar, limoncello liqueur, lemon zest, Price: $9
Dish Name: Caprese Arancini, Description: Crispy risotto balls stuffed with mozzarella, cherry

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', 'Dish Name: Truffle Gnocchi al Tartufo, Description: Potato gnocchi in a decadent truffle cream sauce, garnished with Parmigiano Reggiano, Ingredients: potato gnocchi, truffle cream sauce, Parmigiano Reggiano, Price: $10\nDish Name: Limoncello Tiramisu, Description: A refreshing twist on the classic tiramisu, infused with limoncello liqueur and topped with lemon zest, Ingredients: ladyfingers, mascarpone cheese, eggs, sugar, limoncello liqueur, lemon zest, Price: $9\nDish Name: Caprese Arancini, Description: Crispy risotto balls stuffed with mozzarella, cherry tomatoes, and basil, served with marinara dipping sauce, Ingredients: risotto, mozzarella, cherry tomatoes, basil, marinara sauce, Price: $4')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed: None
Is Task Completed: False
